In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [33]:
%pkmn= pd.read_csv('300k.csv')

UsageError: Line magic function `%pkmn` not found.


In [34]:
%pk = pkmn.convert_objects(convert_numeric=True).dropna()

UsageError: Line magic function `%pk` not found.


In [35]:
from imblearn.combine import SMOTEENN

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

train = pd.read_csv('../final project/300k.csv', low_memory=False)

In [4]:
train = train.drop(['appearedHour', 'appearedMinute', 'appearedDay', 'appearedMonth', 'appearedYear'],1)
#Convert appearedLocalTime string to DateTime
train['appearedLocalTime'] =  pd.to_datetime(train['appearedLocalTime'], format='%Y-%m-%dT%H:%M:%S')        #Note that %y is a 2digit, while %Y is 4digits for the year
#Now reinstate the appeared Hour/Minute/Day/Month/Year, then drop appearedLocalTime
train['appearedHour'] = train['appearedLocalTime'].dt.hour
train['appearedMinute'] = train['appearedLocalTime'].dt.minute
train['appearedDay'] = train['appearedLocalTime'].dt.day
train['appearedMonth'] = train['appearedLocalTime'].dt.month
train['appearedYear'] = train['appearedLocalTime'].dt.year
train = train.drop(['appearedLocalTime'],1)
#Now use 1-of-K encoding using pd.get_dummies()
Hour = pd.get_dummies(train.appearedHour, drop_first=True, prefix='hour')
Minute = pd.get_dummies(train.appearedMinute, drop_first=True, prefix='minute')
Day = pd.get_dummies(train.appearedDay, drop_first=True, prefix='day')
Month = pd.get_dummies(train.appearedMonth, drop_first=True, prefix='month')
Year = pd.get_dummies(train.appearedYear, drop_first=True, prefix='year')
train = train.join(Hour)         #To avoid dummy variable trap
train = train.join(Minute)
train = train.join(Day)
train = train.join(Month)
train = train.join(Year)
#Now we drop the appearedTimeX feature
train = train.drop(['appearedHour', 'appearedMinute', 'appearedDay', 'appearedMonth', 'appearedYear'],1)

In [5]:
#Converting appearedTimeofDay into ordinal
time_mapping = {"morning": 0, "afternoon": 1, "evening": 2, "night": 3}
train['appearedTimeOfDay'] = train['appearedTimeOfDay'].map(time_mapping)

In [6]:
Terr = pd.get_dummies(train.terrainType, drop_first=True, prefix='terr')

In [7]:
City = pd.get_dummies(train.city, drop_first=True, prefix='city')
train = train.join(City)         #To avoid dummy variable trap
#Now we drop the city feature
train = train.drop(['city'],1)

In [8]:
train.continent[train['continent']=='America/Indiana']='America'
train.continent[train['continent']=='America/Kentucky']='America'
train.continent[train['continent']=='Pacific']='Australia'
train.continent[train['continent']=='Atlantic']='Europe'
train.continent[train['continent']=='America/Argentina']='CentralAmerica'
#Then change them to dummies
Continent = pd.get_dummies(train.continent, drop_first=True, prefix='continent')
train = train.join(Continent)         #To avoid dummy variable trap
#Now we drop the continent feature
train = train.drop(['continent'],1)

In [9]:
train['weather'].value_counts()
train['weatherIcon'].value_counts()             #These weather icons are based on time of day as well, making me inclined to not use them.
Weather = pd.get_dummies(train.weather, drop_first=True, prefix='weather')
train = train.join(Weather)         #To avoid dummy variable trap
#Now we drop both weather features
train = train.drop(['weatherIcon', 'weather'],1)

In [10]:
train.loc[(train['windBearing'] >= 337.5), 'windBearing'] = 0
train.loc[(train['windBearing'] < 22.5), 'windBearing'] = 0
train.loc[(train['windBearing'] >= 22.5) & (train['windBearing'] < 67.5), 'windBearing'] = 1
train.loc[(train['windBearing'] >= 67.5) & (train['windBearing'] < 112.5), 'windBearing'] = 2
train.loc[(train['windBearing'] >= 112.5) & (train['windBearing'] < 157.5), 'windBearing'] = 3
train.loc[(train['windBearing'] >= 157.5) & (train['windBearing'] < 202.5), 'windBearing'] = 4
train.loc[(train['windBearing'] >= 202.5) & (train['windBearing'] < 247.5), 'windBearing'] = 5
train.loc[(train['windBearing'] >= 247.5) & (train['windBearing'] < 292.5), 'windBearing'] = 6
train.loc[(train['windBearing'] >= 292.5) & (train['windBearing'] < 337.5), 'windBearing'] = 7
#Now make them into dummies
WindBearing = pd.get_dummies(train.windBearing, drop_first=True, prefix='windBearing')
train = train.join(WindBearing)         #To avoid dummy variable trap
#Now we drop the wind direction feature
train = train.drop(['windBearing'],1)

In [11]:
def OnlyPositiveTime(x):
    if x<0:
        return x+1440                   #Where 1440 = minutes per day
    else:
        return x
    
def OnlyNegativeTime(x):
    if x>0:
        return x-1440                   #Where 1440 = minutes per day
    else:
        return x

In [12]:
SunriseHour = pd.get_dummies(train.sunriseHour, drop_first=True, prefix='sunriseHour')
SunriseMinute = pd.get_dummies(train.sunriseMinute, drop_first=True, prefix='sunriseMinute')
SunsetHour = pd.get_dummies(train.sunsetHour, drop_first=True, prefix='sunsetHour')
SunsetMinute = pd.get_dummies(train.sunsetMinute, drop_first=True, prefix='sunsetMinute')
train = train.join(SunriseHour)         #To avoid dummy variable trap
train = train.join(SunriseMinute)
train = train.join(SunsetHour)
train = train.join(SunsetMinute)
#Now we drop the sunrise/set time features
train = train.drop(['sunriseHour', 'sunriseMinute', 'sunsetHour', 'sunsetMinute'],1)
train['sunriseMinutesMidnight'].apply(OnlyPositiveTime)
train['sunsetMinutesMidnight'].apply(OnlyPositiveTime)
#Make sure that each sighting's minutes since sunrise (sunriseMinutesSince) is positive & that sunsetMinutesBefore is negative
train['sunriseMinutesSince'].apply(OnlyPositiveTime)
train['sunsetMinutesBefore'].apply(OnlyNegativeTime)

0         -196
1         -196
2         -266
3          -77
4          -77
5        -1070
6        -1070
7         -188
8         -196
9         -239
10        -239
11        -239
12        -188
13        -196
14        -188
15         -77
16        -196
17        -101
18        -196
19       -1070
20        -249
21        -188
22        -196
23        -105
24        -196
25        -188
26         -77
27        -105
28        -631
29        -196
          ... 
295991   -1265
295992   -1265
295993    -246
295994    -246
295995    -251
295996   -1167
295997   -1310
295998   -1294
295999   -1145
296000   -1335
296001    -231
296002   -1356
296003    -191
296004    -251
296005   -1335
296006   -1310
296007    -252
296008    -227
296009   -1321
296010   -1294
296011   -1278
296012   -1335
296013    -183
296014    -183
296015    -234
296016   -1337
296017    -227
296018   -1286
296019   -1265
296020    -251
Name: sunsetMinutesBefore, Length: 296021, dtype: int64

In [13]:
train = train.drop(['urban', 'suburban', 'midurban', 'rural'],1)
train.loc[train['population_density'] < 200, 'population_density'] = 0
train.loc[(train['population_density'] >= 200) & (train['population_density'] < 400), 'population_density'] = 1
train.loc[(train['population_density'] >= 400) & (train['population_density'] < 800), 'population_density'] = 2
train.loc[train['population_density'] > 800, 'population_density'] = 3
#Just changing the name to show that I processed
train.rename(columns={'population_density' : 'Urbanity'}, inplace = True)

In [14]:
PokestopDistance = pd.to_numeric(train['pokestopDistanceKm'], errors='coerce')
temporary = pd.concat([train, PokestopDistance], axis=1)
#This ends up dropping 39 instances. I'll find out what is causing the NaN's later (Note: errors='coerce' made them NaN's)
train = temporary.dropna()

In [15]:
row_ids = train[train['class'] != train.pokemonId].index        #This yields an empty set --> identical columns
#So now drop one of them and keep the other (for now) to use as the labels
train.drop(['class'],1)

,pokemonId,latitude,longitude,_id,cellId_90m,cellId_180m,cellId_370m,cellId_730m,cellId_1460m,cellId_2920m,...,sunsetMinute_51,sunsetMinute_52,sunsetMinute_53,sunsetMinute_54,sunsetMinute_55,sunsetMinute_56,sunsetMinute_57,sunsetMinute_58,sunsetMinute_59,pokestopDistanceKm
0,16,20.525745,-97.460829,NTgxMDkzOTk4MTM5MjUwMjIzNw==,9645139108510564000,9645139108711890000,9645139108443455000,9645139109517197000,9645139113812165000,9645139130992034000,...,0,0,0,0,0,0,0,0,0,0.081776
1,133,20.523695,-97.461167,OTQ1NDgzODc1MjM3NDEzMTI2MQ==,9645139109852742000,9645139109785633000,9645139110590940000,9645139109517197000,9645139113812165000,9645139130992034000,...,0,0,0,0,0,0,0,0,0,0.195622
2,16,38.903590,-77.199780,NTQ0OTQ0NDA1Nzg2ODg3OTg2OQ==,9923201472785285000,9923201472986612000,9923201473791918000,9923201477013144000,9923201481308110000,9923201498487980000,...,0,0,0,0,0,0,0,0,0,0.338602
3,13,47.665903,-122.312561,NTU2MTU1NDM4NzA2MDk1MDcxNw==,6093392705025474600,6093392705092583400,6093392705897889800,6093392702676664300,6093392715561566200,6093392767101173800,...,0,0,0,0,0,0,0,0,0,0.109479
4,133,47.666454,-122.311628,MTY2ODg4MTAzMTczMDE0MTUwNTM=,6093392707709829100,6093392707776938000,6093392708045373400,6093392711266598900,6093392715561566200,6093392767101173800,...,0,0,0,0,0,0,0,0,0,0.040364
5,21,-31.954980,115.853609,MTA4NTIwMDE5MjEzOTQ0ODEzNDE=,3040698123425939500,3040698123224612900,3040698123493048300,3040698122419306500,3040698118124339200,3040698100944470000,...,0,0,0,0,0,0,0,0,0,0.011718
6,66,-31.954245,115.852038,NzMxNzg1MTY2MTI0NzQyNDU4OQ==,3040674303268880400,3040674303335989200,3040674303604424700,3040674302530682900,3040674306825650200,3040674324005519400,...,0,0,0,0,0,0,0,0,0,0.040814
7,27,26.235257,-98.197591,MTIzNDcyNjA3MDA4OTA2ODkwMDU=,9684323730997641000,9684323731198968000,9684323731467403000,9684323728246178000,9684323715361276000,9684323663821668000,...,0,0,0,0,0,0,0,0,0,0.958176
8,35,20.525554,-97.458800,MzcwMjg0NjQwNTA0MTk3MDkyNQ==,9645138740753990000,9645138740955316000,9645138741223752000,9645138740150010000,9645138735855043000,9645138718675173000,...,0,0,0,0,0,0,0,0,0,0.116773
9,19,32.928558,-84.340278,NzU5MDI0NzQ4NjYyNjIxMTIyOQ==,9868369829440782000,9868369829373673000,9868369829642109000,9868369828568367000,9868369832863334000,9868369815683465000,...,0,0,0,0,1,0,0,0,0,1.300859


In [16]:
train_features = train.drop(['pokemonId'],1)
train_labels = train['pokemonId']
X_train, X_test, Y_train, Y_test = train_test_split(train_features, train_labels, train_size = 0.7, random_state = 46)
X_train.shape, Y_train.shape, X_test.shape

((207187, 537), (207187,), (88795, 537))

In [17]:
model = DecisionTreeClassifier()
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
acc_9 = round(accuracy_score(Y_test, Y_pred)*100, 2)

MemoryError: 